In [ ]:
# Nombre: Joan Alonzo Machado Domínguez
# https://colab.research.google.com/drive/1-J4MSh4T9BQqRT-LwlLs-W3JNOC4C2sJ?usp=sharing
# Actividad Guiada 2

Tecnica de programacion dinamica


In [ ]:
#Tecnica de programacion dinamica

def Precios(TARIFAS):
  #total de Nodos
  N = len(TARIFAS[0])

  #inicializacion de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N ]
  RUTA = [ [""]*N for i in [""]*N ]

  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i

      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
          MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j])
          RUTA[i][j] = k
          PRECIOS[i][j] = MIN

  return PRECIOS, RUTA

In [ ]:
#ruta con la mejor opcion intermedia para ir de un nodo a otro
def calcular_ruta(RUTA, desde, hasta):
  if desde==hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta]) ) + ', ' + str(RUTA[desde][hasta])

  print("\nLa ruta es:")
  calcular(RUTA, 0, 6)

Ramificacion y poda



In [ ]:
#ramificacion y poda
COSTES = [[11, 12, 18, 40],[14, 15, 13, 22],[11, 17, 19, 23],[17, 14, 20, 28]]

def valor(S, COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i]
  return VALOR

valor((0, 1, 2, 3), COSTES)

73

In [ ]:
import itertools
from itertools import permutations

#fuerza bruta
#calcular_tiempo
def fuerza_bruta(COSTES):
  #representacion de la solucion sera una tupla donde cada valor en la coordenada i-sima es la tarea asignado al agente i
  # ejemplo (1,2,3,4) tiene valor 11+15+19+28=73
  # ¿cuantas posibilidades hay? n! -> complejidad factorial(exponencial)
  # con dimiension 11 se va a l minuto ejecucion

  mejor_valor = 10e10
  mejor_solucion = ()

  for s in list(itertools.permutations(range(len(COSTES)))):
    #print(s, valor(s, COSTES))
    valor_tmp = valor(s, COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s

  print("la mejor solucion es: "+ str(mejor_solucion) +" con valor: "+ str(mejor_valor))

fuerza_bruta(COSTES)

la mejor solucion es: (0, 3, 1, 2) con valor: 61


In [ ]:
#funcion para estimar una cota inferior para una solucion parcial
#costo inferior para soluciones parciales
#(1, 3, )se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S, COSTES):
  VALOR=0
  #Valores establecidos
  for i in range(len(S)):
    VALOR+=COSTES[i][S[i]]

  #estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR+=min([ COSTES[j][i] for j in range(len(S), len(COSTES)) ])
  
  return VALOR

CI((2,0,), COSTES)

69

In [ ]:
#funcion para ramificar

#Genera tantos hijos como posibilidades haya para el siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({ 's':NODO +(i,) })
  return HIJOS

In [ ]:
#calcular el tiempo

def ramificacion_y_poda(COSTES):
  #construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas)
  #nodos del grafo {s:(1,2),CI:3,CS:5}

  DIMENSION = len(COSTES)
  MEJOR_SOLUCION = tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION, COSTES)

  NODOS=[]
  NODOS.append({ 's':(), 'ci':CI((),COSTES)})

  iteracion = 0
  while(len(NODOS) > 0):
    iteracion+=1

    nodo_prometedor = [min(NODOS, key=lambda x:x['ci'])][0]['s']

    #ramificacion, se generan los hijos
    HIJOS = [{'s':x['s'], 'ci':CI(x['s'], COSTES)} for x in crear_hijos(nodo_prometedor, DIMENSION)]

    #revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) > 0:
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
    
    #poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    #añadimos los hijos
    NODOS.extend(HIJOS)

    #eliminamos el nodo ramificado
    NODOS = [x for x in NODOS if x['s'] != nodo_prometedor]

  print("la solucion final es: "+ str(MEJOR_SOLUCION) + " en "+ str(iteracion) + "iteraciones, para dimension: "+ str(DIMENSION))

ramificacion_y_poda(COSTES)

la solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}] en 14iteraciones, para dimension: 4


#### Actividad Individual

- ¿Que complejidad tiene el algoritmo por fuerza bruta?
- Generar matrices con valores aleatorios de mayores dimensiones (5,6,7...) y ejecutar ambos algoritmos
- ¿A partir de que dimensión el algoritmo por fuerza bruta deja de ser una opción?
- ¿Hay algún valor de la dimensión a partir de la cual el algoritmo de ramificación y poda deja de ser una opción válida?